In [2]:
import pandas as pd 
import numpy as np 
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt 

In [3]:
#df features 
df_features = pd.read_csv('/Users/arasdirekoglu/Documents/Github/Fault-Detection-SECOM-2/secom.data', delimiter=' ', header=None, na_values=['NaN'])
df_features.columns = ['feature'+str(x+1) for x in range(len(df_features.columns))] 

#df target 
df_target = pd.read_csv('/Users/arasdirekoglu/Documents/Github/Fault-Detection-SECOM-2/secom_labels.data', delimiter= ' ', header=None, na_values={'NaN'})
df_target.columns = ['status', 'timestamp']
df_target['timestamp'] = pd.to_datetime(df_target['timestamp'], dayfirst=True)

#concat both to df
df = pd.concat([df_features, df_target], axis=1) 

Attempt KNN Imputation 

In [4]:
#Number of Missing Values BEFORE KNN Imputation
df_features.isna().any(axis=1).sum()

1567

In [5]:
#KNN Imputation
knn = KNNImputer()
knn.fit(df_features)
data_knn=pd.DataFrame(knn.fit_transform(df_features), columns=df_features.columns)

In [6]:
#Number of Missing Values AFTER KNN Imputation
data_knn.isna().any(axis=1).sum()

0

Defition & Test of Function

In [7]:
#Definition of Function
def knnimputation(x):
    knn = KNNImputer()
    knn.fit(x)
    df=pd.DataFrame(knn.fit_transform(x), columns=x.columns)
    return df  

In [8]:
#Number of Missing Values BEFORE KNN Imputation
df_features.isna().any(axis=1).sum()

1567

In [9]:
#Test of function 
df_test_function = knnimputation(df_features)
df_test_function.isna().any(axis=1).sum()

0

Test of Function with a completely different Data Set

In [10]:
#Read the Data
df_test = pd.read_csv('testdata.csv')
df_test =  df_test[['Age', 'Fare', 'Pclass']]
df_test.head()

,Age,Fare,Pclass
0,22.0,7.2500,3
1,38.0,71.2833,1
2,26.0,7.9250,3
3,35.0,53.1000,1
4,35.0,8.0500,3


In [11]:
df_test.isna().any(axis=1).sum()

177

In [12]:
df_test_knn = knnimputation(df_test)
df_test_knn.isna().any(axis=1).sum()

0